In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  # !pip install tf-nightly # Removed tf-nightly installation
  pass
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load and preprocess data
# Define column names for the dataset
COLUMN_NAMES = ['label', 'message']

# Load training and test data
train_data = pd.read_csv('train-data.tsv', sep='\t', header=None, names=COLUMN_NAMES)
test_data = pd.read_csv('valid-data.tsv', sep='\t', header=None, names=COLUMN_NAMES)

# Convert labels to binary: ham -> 0, spam -> 1
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

# Split data into features and labels
train_messages = train_data['message'].values
train_labels = train_data['label'].values
test_messages = test_data['message'].values
test_labels = test_data['label'].values

# Create text vectorization layer
max_vocab_size = 10000
max_sequence_length = 100

vectorize_layer = keras.layers.TextVectorization(
    max_tokens=max_vocab_size,
    output_mode='int',
    output_sequence_length=max_sequence_length
)

# Adapt the vectorization layer to the training messages
vectorize_layer.adapt(train_messages)

# Build the model
model = keras.Sequential([
    keras.layers.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    keras.layers.Embedding(
        input_dim=max_vocab_size,
        output_dim=16,
        input_length=max_sequence_length
    ),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# Train the model
history = model.fit(
    train_messages,
    train_labels,
    epochs=20,
    validation_data=(test_messages, test_labels),
    verbose=2
)

In [ ]:
# Fixed prediction function
def predict_message(pred_text):
    # Pass the raw input string directly to the model for prediction
    # Keras expects input as a numpy array of object dtype for strings
    input_data = np.array([pred_text], dtype=object)
    prediction = model.predict(input_data)
    spam_prob = float(prediction[0][0])
    label = 'spam' if spam_prob > 0.5 else 'ham'
    return [spam_prob, label]

# Test the function
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
